# Analysis of a Ventilation Air Exhaust System  

**Imports**

Module `hvac.fluids.fluid` will raise a `CoolPropWarning` each time one of the properties of a fluid cannot be determined. Especially, the surface tension property, which is only defined for a two-phase mixture, can be the reason why this warning is given frequently. But, as we don't need this property in fluid flow calculations, it is preferable to ignore this warning here to avoid unnecessary warning messages.

In [1]:
import warnings
from hvac.fluids import CoolPropWarning
warnings.filterwarnings('ignore', category=CoolPropWarning)

In [2]:
import pandas as pd

In [3]:
from hvac import Quantity
from hvac.fluids import Fluid
from hvac.fluid_flow import (
    DuctNetwork,
    circular_duct_schedule,
    Duct,
    Circular
)
import hvac.fluid_flow.fittings.duct as duct_fittings

In [4]:
Q_ = Quantity

In [5]:
from IPython.display import display, HTML

## Configuration of the Ventilation Air Exhaust System

File `network_scheme.pdf`, in the same folder as this notebook, shows a scheme that was prepared for analyzing the duct network. With this scheme at hand, we can create a network configuration table in a spreadsheet program, and then save this spreadsheet with the .csv extension (i.e., a comma-separated-values file). In the code cell below, the network configuration file is loaded into a Pandas DataFrame object for displaying on screen: 

In [6]:
network_config_table = pd.read_csv('network_config.csv')
display(HTML(network_config_table.to_html()))

,conduit_ID,start_node_ID,end_node_ID,loop_ID,length,volume_flow_rate,nominal_diameter,fixed_pressure_difference
0,D3,N1,N2,L3,25.5,-222.0,160.0,NaN
1,D4,N2,N7,"(L3, L4)",2.2,-61.0,125.0,NaN
2,D5,N2,N8,"(L4, L5)",4.5,-61.0,125.0,NaN
3,D6,N2,N9,"(L5, L6)",5.9,-50.0,125.0,NaN
4,D7,N2,N10,L6,25.4,-50.0,125.0,NaN
5,D2,N1,N6,"(L2, L3)",3.5,-50.0,125.0,NaN
6,D8,N1,N3,L2,20.9,150.0,125.0,NaN
7,D9,N3,N5,"(L1, L2)",10.7,-100.0,125.0,NaN
8,D10,N3,N4,L1,5.5,50.0,125.0,NaN
9,D11,N4,N5,L1,NaN,NaN,NaN,0.0


The network configuration table has been prepared for analyzing the duct network using the Hardy Cross method. The fact that we are analyzing an existing duct system implies that we know the length and diameter of each duct in the network. The volume flow rates through the ducts are in fact initially guessed values, which could be e.g., design values demanded by a certain ventilation standard. When volume flow rates are assigned to ducts, care must be taken to respect the physical law of conservation of mass in the nodes of the network.

From the scheme, it can be seen that the network contains 6 loops. With each loop a positive loop sense is associated, which is conventionally chosen to be the clock-wise sense. In the duct network, all air is exhausted through node N1 and air enters the duct system through exhaust valves at nodes N4 to N10. With this information, the sense of the airflow direction in each duct can be determined. In the Hardy Cross method, the sense of direction of the air flow in a duct is coupled to the sense of the loop's direction to which the duct belongs. E.g., in duct D10 the sense of the airflow direction matches with the sense of its loop L1. As such, we've entered the volume flow rate through duct D10 with a positive sign in the network configuration file. On the other hand, the sense of airflow direction in duct D7 is opposite to the sense of its loop L6, and therefore we've entered the volume flow rate through duct D7 with a minus sign in the network configuration file. 

A duct can belong to 2 loops. E.g., duct D4 belongs to loop L3 and to loop L4. In the network configuration file we've entered the loops L3 and L4 between parentheses and separated by a comma (note: the parentheses should not be forgotten!). The sign of the volume flow rate is referred to the sense of the first entered loop. In this example, this is loop L3. As the sense of airflow direction in duct D4 is opposite to the sense of loop L3, we've entered the volume flow rate through duct D4 with a minus sign.

The column names cannot be chosen freely, but the order of the columns is less important, as the csv-file is read using the `csv.DictReader` class. Note that each duct must have a start node and an end node. The program needs this information to find out how ducts are interconnected in the network.

Note that all loops in this example are actually open loops. E.g. in loop L1, there is, in reality, no duct present between nodes N4 and N5. However, in the network configuration file we've entered a duct D11 between nodes N4 and N5. Such a fictitious duct, used to close a loop, is also called a "pseudo duct". It has no length, no diameter, and there is no air flow through it. In the network configuration file, we leave these cells empty, which is the reason why NaN ("Not a Number") was filled in by Pandas, when it read the csv-file from disk into `network_config_table` (a `DataFrame` object). On the other hand, there can be a fixed pressure difference between the start and end node of a pseudo duct. In this example, nodes N4 to N10 are all at the same pressure (atmospheric pressure), so we've entered 0 in column *fixed_pressure_difference* for each of the pseudo ducts present in the network. For real ducts, column *fixed_pressure_difference* has no meaning, and we should leave the cells under this column empty. Should there be a fixed pressure difference between the start and end node of a pseudo duct that isn't zero, we must take care to also add the appropriate sign to this fixed pressure difference. E.g. suppose that node N4 is at higher pressure than node N5. In that case, air would flow from node N4 to node N5, and the sense of direction of this fictitious air flow would be opposite to the sense of loop L1, which is why a minus sign should be added to the value of the fixed pressure difference between node N4 and node N5.  

The configuration of the duct network has already been made in a spreadsheet program, and the spreadsheet table has been saved in csv-format to the file `network_config.csv`. Now, we need to create a `DuctNetwork` object. Then, the configuration file can be passed to this object. However, before the `DuctNetwork` object can be created, we first need to define the fluid that will flow in this duct system. Obviously, in this case, the fluid will be air. We will use dry air at standard conditions:

In [7]:
Air = Fluid('Air')
standard_air = Air(T=Q_(20, 'degC'), P=Q_(101_325, 'Pa'))

Now, we can create the `DuctNetwork` object:

In [8]:
duct_network = DuctNetwork.create(
    ID='air_exhaust_system',
    fluid=standard_air,
    wall_roughness=Q_(0.09, 'mm'),
    schedule=circular_duct_schedule,
    start_node_ID='N1',
    units={'volume_flow_rate': 'm ** 3 / hr'}
)

To configure the duct network with ducts, we now load our configuration file into this object:

In [9]:
duct_network.load_from_csv('network_config.csv')

We can take a look at the duct network configuration by getting the duct table (which is actually a Pandas DataFrame object):

In [10]:
duct_table = duct_network.get_duct_table()
display(HTML(duct_table.to_html()))

,duct ID,L [m],Deq [mm],width [mm],height [mm],V [m³/h],v [m/s],Re,Δp-dyn. [Pa]
0,D3,25.5,160.0,NaN,NaN,-222.0,3.067048,32468.911623,22.179468
1,D4,2.2,125.0,NaN,NaN,-61.0,1.380758,11419.696484,0.626118
2,D5,4.5,125.0,NaN,NaN,-61.0,1.380758,11419.696484,1.280697
3,D6,5.9,125.0,NaN,NaN,-50.0,1.131768,9360.406954,1.182929
4,D7,25.4,125.0,NaN,NaN,-50.0,1.131768,9360.406954,5.092609
5,D2,3.5,125.0,NaN,NaN,-50.0,1.131768,9360.406954,0.701737
6,D8,20.9,125.0,NaN,NaN,150.0,3.395305,28081.220863,29.752488
7,D9,10.7,125.0,NaN,NaN,-100.0,2.263537,18720.813909,7.335677
8,D10,5.5,125.0,NaN,NaN,50.0,1.131768,9360.406954,1.102730


At this point, the duct network is configured, but the ducts don't contain any fittings. The following step is to add the fittings to each duct of the network.

## Adding Fittings to Ducts

### Node N1

As shown on the scheme, at node N1 we have a four-legged fitting, where air from duct D8, duct D2, and duct D3 joins and is exhausted through a common duct D1. Note that this common duct D1 was not added to the network configuration file; it is considered to be external to the duct system we want to analyze. In this system all air is exhausted through node N1, which lies on the boundary of our system. To represent the four-legged fitting, we will use 2 tees. One tee connects duct D2 and duct D3 with common duct D1, while the other tee connects duct D2 and duct D8 with the common duct D1. First, we create and configure the common duct:   

In [11]:
duct_D1 = Duct.create(
    length=Q_(1, 'm'),
    wall_roughness=Q_(0.09, 'mm'),
    fluid=standard_air,
    cross_section=Circular.create(schedule=circular_duct_schedule),
    volume_flow_rate=Q_(422, 'm ** 3 / hr'),
    specific_pressure_drop=Q_(0.7, 'Pa / m')
)

We created the common duct D1 without specifying any diameter, but we specified the volume flow rate through the duct and the specific pressure drop, also indicating that the cross-section of the duct must be circular and that its diameter must be selected from a commercially available duct schedule, in this case `circular_duct_schedule`, which is already included in the `fluid_flow` package, and that we imported at the beginning of this notebook. From this, the diameter of the common duct can be determined:  

In [12]:
print(duct_D1.cross_section.internal_diameter)

200.0 millimeter


Now we will add the 2 tees at node N1. The first tee will connect duct D2 and duct D3 with common duct D1. Duct D2 will be connected to the straight leg of the tee and duct D3 to the branch leg of the tee:

In [13]:
tee_N1a = duct_fittings.ConvergingJunctionA10B(
    duct_b=duct_network.conduits['D3'],
    duct_s=duct_network.conduits['D2'],
    duct_c=duct_D1,
    ID='TEE-N1-1'
)

As the air from the branch leg and the air from the straight leg joins at the common leg of the tee, the tee is a converging junction. 'A10B' refers to the specific tee fitting. The letter 'A' refers to appendix A of SMACNA *HVAC Systems Duct Design Manual*, and '10B' refers to the fitting loss coefficient table in this appendix.

The fitting has now been created, but must still be added to the right ducts. We add the pressure loss across the straight and common leg to straight duct D2 and the pressure loss across the branch leg and common leg to branch duct D3. Therefore, the pressure loss coefficient of the tee must be referred, on one hand, to the straight leg (`zeta_s`), and on the other hand to the branch leg (`zeta_b`). To add the tee to branch duct D3, we write:  

In [14]:
duct_network.conduits['D3'].add_fitting(
    zeta=tee_N1a.zeta_b, 
    ID=tee_N1a.ID
)

And to add the tee to straight duct D2, we write:

In [15]:
duct_network.conduits['D2'].add_fitting(
    zeta=tee_N1a.zeta_s, 
    ID=tee_N1a.ID
)

The same must now be done to connect duct D8 and duct D2 with common duct D1. Again, duct D2 is the straight duct, and duct D8 can be considered as the branch duct.

In [16]:
tee_N1b = duct_fittings.ConvergingJunctionA10B(
    duct_b=duct_network.conduits['D8'],
    duct_s=duct_network.conduits['D2'],
    duct_c=duct_D1,
    ID='TEE-N1-2'
)

duct_network.conduits['D8'].add_fitting(
    zeta=tee_N1b.zeta_b,
    ID=tee_N1b.ID
)

# duct_network.conduits['D2'].add_fitting(
#     zeta=tee_N1b.zeta_s,
#     ID=tee_N1b.ID
# )

As we already associated a pressure loss with straight duct D2, when connecting it using `tee_N1a`, we commented out the addition of `tee_N1b` to this same duct.

### Duct D2

Duct D2 contains 2 elbows and 1 exhaust valve.

To add the 2 elbows, we can use a for-loop:

In [17]:
for n in [1, 2]:
    elbow = duct_fittings.ElbowA7A(
        duct=duct_network.conduits['D2'],
        R_on_D=Q_(1, 'frac'),
        theta=Q_(90, 'deg'),
        ID=f'ELB90-D2-{n}'
    )

    duct_network.conduits['D2'].add_fitting(
        zeta=elbow.zeta,
        ID=elbow.ID
    )

The exhaust valve is not included in the duct fittings module. We selected an exhaust valve from a catalog, in which the pressure drop as a function of volume flow rate is specified. For a volume flow rate of 50 m³/h the corresponding pressure drop would be 12 Pa, according to the catalog. To create the exhaust valve, we can use the general `DuctFitting` class. Knowing the volume flow rate and pressure drop, the flow coefficient of the exhaust valve can be calculated, and for this we can use the `FlowCoefficient` class. So, to add the exhaust valve to duct D2, we should write the following code:

In [18]:
exh_valve_D2 = duct_fittings.DuctFitting(
    duct=duct_network.conduits['D2'],
    Av=duct_fittings.FlowCoefficient.get_Av(
        volume_flow_rate=Q_(50, 'm ** 3 / hr'),
        pressure_drop=Q_(12, 'Pa')  # from catalog
    ),
    ID="EXH-VLV-D2"
)

duct_network.conduits['D2'].add_fitting(
    zeta=exh_valve_D2.zeta,
    ID=exh_valve_D2.ID
)

Now we must continue adding fittings to all other ducts in the duct system...To ease this pain a little, we could write a few functions to add frequently used fittings more quickly to a duct.

**Function for adding one or more elbows to a duct:**

In [19]:
def add_elbow(
    conduit_ID: str,
    num: int = 1,
    angle: int = 90
) -> None:
    for n in range(1, num + 1):
        elbow = duct_fittings.ElbowA7A(
            duct=duct_network.conduits[conduit_ID],
            R_on_D=Q_(1, 'frac'),
            theta=Q_(angle, 'deg'),
            ID=f"ELB{angle}-{conduit_ID}-{n}"
        )

        duct_network.conduits[conduit_ID].add_fitting(
            zeta=elbow.zeta, 
            ID=elbow.ID
        )

**Function for adding an exhaust valve to a duct:**

In [20]:
def add_exhaust_valve(
    conduit_ID: str,
    V: float,
    dP: float
) -> None:
    exh_valve = duct_fittings.DuctFitting(
        duct=duct_network.conduits[conduit_ID],
        Av=duct_fittings.FlowCoefficient.get_Av(
            volume_flow_rate=Q_(V, 'm ** 3 / hr'),
            pressure_drop=Q_(dP, 'Pa')  # from catalog
        ),
        ID=f"EXH-VLV-{conduit_ID}"
    )

    duct_network.conduits[conduit_ID].add_fitting(
        zeta=exh_valve.zeta, 
        ID=exh_valve.ID
    )

**Function for adding a multi-junction to a node:**

In [21]:
def add_multi_junction(
    node_ID: str,
    common_conduit_ID: str,
    straight_conduit_ID: str,
    branch_conduit_IDs: list[str]
) -> None:
    for i, branch_conduit_ID in enumerate(branch_conduit_IDs):
        tee = duct_fittings.ConvergingJunctionA10B(
            duct_c=duct_network.conduits[common_conduit_ID],
            duct_b=duct_network.conduits[branch_conduit_ID],
            duct_s=duct_network.conduits[straight_conduit_ID],
            ID=f'TEE-{node_ID}-{i+1}'
        )

        duct_network.conduits[branch_conduit_ID].add_fitting(
            zeta=tee.zeta_b, 
            ID=tee.ID
        )

        if i == 0:
            duct_network.conduits[common_conduit_ID].add_fitting(
                zeta=tee.zeta_c, 
                ID=tee.ID
            )

### Duct D3

In [22]:
add_elbow(conduit_ID='D3', num=4)

In [23]:
add_elbow(conduit_ID='D3', angle=45)

### Node N2

In [24]:
add_multi_junction(
    node_ID='N2',
    common_conduit_ID='D3',
    straight_conduit_ID='D7',
    branch_conduit_IDs=['D4', 'D5', 'D6']
)

### Duct D4

In [25]:
add_exhaust_valve(conduit_ID='D4', V=50, dP=12)

### Duct D5

In [26]:
add_elbow(conduit_ID='D5', num=2)

In [27]:
add_exhaust_valve(conduit_ID='D5', V=50, dP=12)

### Duct D6

In [28]:
add_elbow(conduit_ID='D6', num=2)

In [29]:
add_exhaust_valve(conduit_ID='D6', V=50, dP=12)

### Duct D7

In [30]:
add_elbow(conduit_ID='D7', num=2)

In [31]:
add_exhaust_valve(conduit_ID='D7', V=50, dP=12)

### Duct D8

In [32]:
add_elbow(conduit_ID='D8', num=5)

### Node N3

In [33]:
add_multi_junction(
    node_ID='N3',
    common_conduit_ID='D8',
    straight_conduit_ID='D10',
    branch_conduit_IDs=['D9']
)

### Duct D9

In [34]:
add_elbow(conduit_ID='D9')

In [35]:
add_elbow(conduit_ID='D9', angle=45)

In [36]:
add_exhaust_valve(conduit_ID='D9', V=100, dP=18)

### Duct D10

In [37]:
add_elbow(conduit_ID='D10', num=2)

In [38]:
add_exhaust_valve(conduit_ID='D10', V=50, dP=12)

## Duct Network Overview

Once all fittings have been added to the ducts, the configuration of the duct network is finished. We can take a look again at the duct table to see that by adding the fittings to the ducts, the pressure losses across the ducts have been changed. It is also possible to get a table with an overview of all the fittings present in the network, together with their pressure loss coefficient (zeta) and associated pressure drop. Finally, we can also get a table showing the different flow paths in the duct system.

### Duct Table

In [39]:
duct_table = duct_network.get_duct_table()
display(HTML(duct_table.to_html()))

,duct ID,L [m],Deq [mm],width [mm],height [mm],V [m³/h],v [m/s],Re,Δp-dyn. [Pa]
0,D3,25.5,160.0,NaN,NaN,-222.0,3.067048,32468.911623,35.700097
1,D4,2.2,125.0,NaN,NaN,-61.0,1.380758,11419.696484,19.151601
2,D5,4.5,125.0,NaN,NaN,-61.0,1.380758,11419.696484,20.311413
3,D6,5.9,125.0,NaN,NaN,-50.0,1.131768,9360.406954,13.213231
4,D7,25.4,125.0,NaN,NaN,-50.0,1.131768,9360.406954,17.432056
5,D2,3.5,125.0,NaN,NaN,-50.0,1.131768,9360.406954,17.572880
6,D8,20.9,125.0,NaN,NaN,150.0,3.395305,28081.220863,48.156483
7,D9,10.7,125.0,NaN,NaN,-100.0,2.263537,18720.813909,31.073871
8,D10,5.5,125.0,NaN,NaN,50.0,1.131768,9360.406954,13.442177


### Fitting Table

In [40]:
fitting_table = duct_network.get_fitting_table()
display(HTML(fitting_table.to_html()))

,conduit ID,fitting ID,zeta,pressure drop [Pa]
0,D3,TEE-N1-1,1.022194,5.791335
1,D3,ELB90-D3-1,0.22,1.246430
2,D3,ELB90-D3-2,0.22,1.246430
3,D3,ELB90-D3-3,0.22,1.246430
4,D3,ELB90-D3-4,0.22,1.246430
5,D3,ELB45-D3-1,0.132,0.747858
6,D3,TEE-N2-1,0.3522522522522523,1.995717
7,D4,TEE-N2-1,0.578863,0.664683
8,D4,EXH-VLV-D4,15.554718,17.860800
9,D5,TEE-N2-2,0.578863,0.664683


### Flow Paths

In [41]:
flow_path_table = duct_network.get_flow_path_table()
display(HTML(flow_path_table.to_html()))

,path,Δp-elev. [Pa],Δp-dyn. [Pa],Δp-tot. [Pa],Δp-deficit [Pa]
0,D3|D4,0.0,54.851699,54.851699,24.378656
1,D2,0.0,17.572880,17.572880,61.657474
2,D8|D9,0.0,79.230355,79.230355,0.000000
3,D8|D10,0.0,61.598660,61.598660,17.631694
4,D3|D5,0.0,56.011510,56.011510,23.218845
5,D3|D6,0.0,48.913328,48.913328,30.317027
6,D3|D7,0.0,53.132153,53.132153,26.098201


The flow path table identifies a flow path by listing the duct ID's that belong to this path. Next to each path, 4 different pressure differences are listed:
- 'Δp-elev.' refers to the pressure difference between the start and end node of a path due to elevation, i.e., due to the height difference between the end and start node of the path (aka as gravity or chimney effect).
- 'Δp-dyn.' refers to the pressure loss between the start and end node of a path due to fluid flow and fitting losses, i.e., the dynamic pressure difference between the start and end node of the path.
- 'Δp-tot.' is the sum of the elevation pressure difference and the dynamic pressure difference. It is also the difference between the total pressure at the start node and the total pressure at the end node of the path, plus any pressure difference added by pumps or fans in the flow path.
- 'Δp-deficit.' is the difference between the total pressure difference of the critical flow path and the total pressure difference of the path under consideration. The critical flow path is the flow path with the greatest total pressure difference and for which the pressure difference deficit is evidently zero. 

The physical law of conservation of energy requires that the pressure difference deficit along each flow path must be zero, as air is taken up by the exhaust air system from the indoor environment (nodes N4 to N10 are all at the same pressure of the indoor environment) and each flow path ends in the same outdoor environment, which is at atmospheric pressure. So, the same pressure difference must exist across each flow path. It is obvious here that with the initial volume flow rates that we've entered into the network configuration table, the law of conservation of energy is not being obeyed. Consequently, the purpose of analyzing this air exhaust system is to find the real volume flow rates that will pass through each duct of the network.

## Analysis of the Unbalanced Duct Network

To analyze the fully configured duct network, we just need to call method `analyze` on our `DuctNetwork` instance. As already mentioned, the analysis is done with the method of Hardy Cross. This method uses an iterative solving technique. We can change the default maximum number of iterations and the default tolerance (stop criterion) through parameters `i_max` and `tolerance`, but here we'll accept the default settings:

In [42]:
duct_network.analyze()

15

The output displayed above is the number of iterations that were needed to meet the stop criterion. The default value for the tolerance is 1 Pa, which means that the analyzing routine will terminate when the pressure difference across each loop of the network becomes less than 1 Pa. In fact, the law of conservation of energy requires the pressure difference across each loop to be zero (any point along any loop in the network can have only one pressure or energy value). By making the tolerance smaller, the results will become more exact, but the number of iterations to meet the stop criterion will evidently increase. 

**Duct table**<br>
From the duct table we can now read the actual volume flow rates in the air exhaust system: 

In [43]:
duct_table = duct_network.get_duct_table()
display(HTML(duct_table.to_html()))

,duct ID,L [m],Deq [mm],width [mm],height [mm],V [m³/h],v [m/s],Re,Δp-dyn. [Pa]
0,D3,25.5,160.0,NaN,NaN,-213.360277,2.947686,31205.297250,33.130157
1,D4,2.2,125.0,NaN,NaN,-55.499634,1.256255,10389.983139,15.865204
2,D5,4.5,125.0,NaN,NaN,-54.965770,1.244171,10290.039454,16.517585
3,D6,5.9,125.0,NaN,NaN,-54.783031,1.240034,10255.829369,15.831187
4,D7,25.4,125.0,NaN,NaN,-48.111843,1.089029,9006.928519,16.184859
5,D2,3.5,125.0,NaN,NaN,-84.472422,1.912064,15813.924855,49.929123
6,D8,20.9,125.0,NaN,NaN,124.167301,2.810573,23245.129360,33.751931
7,D9,10.7,125.0,NaN,NaN,-70.055318,1.585728,13114.925675,15.540551
8,D10,5.5,125.0,NaN,NaN,54.111983,1.224845,10130.203685,15.719651


**Flow path table**<br>
From the flow path table we can verify that the duct network is now balanced. Each flow path has almost the same total pressure difference, and the pressure difference deficit of each flow path is nearing to zero. 

In [44]:
flow_path_table = duct_network.get_flow_path_table()
display(HTML(flow_path_table.to_html()))

,path,Δp-elev. [Pa],Δp-dyn. [Pa],Δp-tot. [Pa],Δp-deficit [Pa]
0,D3|D4,0.0,48.995360,48.995360,0.933763
1,D2,0.0,49.929123,49.929123,0.000000
2,D8|D9,0.0,49.292482,49.292482,0.636641
3,D8|D10,0.0,49.471582,49.471582,0.457541
4,D3|D5,0.0,49.647741,49.647741,0.281382
5,D3|D6,0.0,48.961344,48.961344,0.967780
6,D3|D7,0.0,49.315016,49.315016,0.614107
